In [1]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import PercentFormatter
import matplotlib.ticker as ticker

import numpy as np
import pandas as pd

import os
import itertools
import subprocess

In [3]:
trace = "../traces/df_AMG_n8_dumpi/"
precision = 1 # 1 is max. precision (1 data point per 1 MPI progress operation)
subprocess.check_call(["./run.sh", str(trace), str(precision)])

app_name = "AMG with 8 MPI processes"

folders = list([
    "dumps/df16",
    "dumps/df32",
    "dumps/df64",
    "dumps/df128",
    "dumps/df256"
])

stats_dfs = list([pd.read_csv(os.path.join(path, "stats.csv")) for path in folders])
tags_dfs = pd.read_csv(os.path.join(folders[0], "tags.csv"))

bucket_numbers = [16, 32, 64, 128, 256]

collision_names = ['collisions_no_wildcard', 'collisions_src_wildcard', 'collisions_tag_wildcard', 'collisions_double_wildcard_max_len']
aggr_collision_names = ['aggr_collisions_no_wildcard', 'aggr_collisions_src_wildcard', 'aggr_collisions_tag_wildcard', 'collisions_double_wildcard_max_len']
collision_depth_names = ['collisions_no_wildcard_max_len', 'collisions_src_wildcard_max_len', 'collisions_tag_wildcard_max_len', 'collisions_double_wildcard_max_len']
collision_prob_names = ['empty_bins_perc_no_wildcard', 'empty_bins_perc_src_wildcard', 'empty_bins_perc_tag_wildcard']

recvs_count_names = [
    ['recv_no_wildcard_sync', 'recv_no_wildcard_async'],
    ['recv_src_wildcard_sync', 'recv_src_wildcard_async'],
    ['recv_tag_wildcard_sync', 'recv_tag_wildcard_async'],
    ['recv_double_wildcard_sync', 'recv_double_wildcard_async']
]

rank_count = stats_dfs[0]['rank'].max() + 1
entries_per_rank = len(stats_dfs[0].loc[stats_dfs[0]['rank'] == 0])

FileNotFoundError: [Errno 2] No such file or directory: 'dumps/df16/stats.csv'

In [ ]:
# Cum. Avg. of collisions per rank
# x = list(range(0, 52))

# fig, axs = plt.subplots(4, 2, sharex=True, sharey=True)

# for (n, pl) in zip(range(0, 8), axs.flat):  
#     for (dfx, siz) in zip(dfs, bucket_numbers):
#         local = dfx.loc[dfx['rank'] == n]
#         lcum = np.cumsum(local['collisions_no_wildcard']) / x
#         lab = str(siz)

#         pl.plot(x, lcum, label=lab)
#         pl.set_title('Rank ' + str(n))
        
#         if n % 4 == 0:
#             pl.set(ylabel='Cum. Avg. of colls.')
#         pl.set(xlabel='Runtime',)

#         pl.label_outer()
#         pl.set_xticks([])
#         pl.yaxis.set_major_locator(ticker.Mafig.supxlabel('Runtime')
# fig.supylabel('Avg. number of collisions')
# fig.legend(labels=[str(i) for i in bucket_numbers], title='N. Bins', loc='lower right') 

# plt.suptitle(app_name)

# plt.tight_layout()xNLocator(integer=True))

# plt.legend(title='N. Bins', borderaxespad=0.1)
# plt.tight_layout()
# plt.show()

In [ ]:
# Aggr. Avg. of collisions per bin
fig, axs = plt.subplots(4, 1, sharex=True, sharey=False)

translation = ["No Wildcard", "MPI_ANY_SRC", "MPI_ANY_TAG", "MPI_ANY_SRC & MPI_ANY_TAG"]

for (ax, label, print_label) in zip(axs.flat, collision_names, translation):
    for (table, bucket_size) in zip(stats_dfs, bucket_numbers):
        ser = pd.Series([0] * entries_per_rank)
        for rank in range(0, rank_count):
            local = table.loc[table['rank'] == rank]
            l_cols = local[label]

            l_cols.reset_index(drop=True, inplace=True)
            ser += l_cols
        
        ax.plot(ser / rank_count / precision, label=str(bucket_size))

    ax.set_ylim(bottom=0)
    ax.set_xticks([])
    ax.set_title(print_label, fontsize=10)
    ax.grid(True)


fig.supxlabel('Runtime')
fig.supylabel('Avg. number of collisions')
fig.legend(labels=[str(i) for i in bucket_numbers], title='N. Bins', loc='lower right') 

plt.suptitle(app_name)

plt.tight_layout()
plt.show()

In [ ]:
# Max. number of collisions per bin
fig, axs = plt.subplots(4, 1, sharex=True, sharey=False)

# translation = ["No Wildcard", "MPI_ANY_SRC", "MPI_ANY_TAG", "MPI_ANY_SRC & MPI_ANY_TAG"]
translation = ["No Wildcard"]

for (ax, label, print_label) in zip(axs.flat, collision_names, translation):
    for (table, bucket_size) in zip(stats_dfs, bucket_numbers):
        ser = pd.Series([0] * entries_per_rank)
        for rank in range(0, rank_count):
            local = table.loc[table['rank'] == rank]
            l_cols = local[label]

            l_cols.reset_index(drop=True, inplace=True)
            ser = np.maximum(ser, l_cols)
        
        ax.plot(ser, label=str(bucket_size))

    ax.set_ylim(bottom=0)
    ax.set_xticks([])
    ax.set_title(print_label, fontsize=10)
    ax.grid(True)


fig.supxlabel('Runtime')
fig.supylabel('Max. number of collisions')
fig.legend(labels=[str(i) for i in bucket_numbers], title='N. Bins', loc='lower right') 

plt.suptitle(app_name)

plt.tight_layout()
plt.show()

In [ ]:
# Max. aggr. number of collisions per bin
fig, axs = plt.subplots(4, 1, sharex=True, sharey=False)

translation = ["No Wildcard", "MPI_ANY_SRC", "MPI_ANY_TAG", "MPI_ANY_SRC & MPI_ANY_TAG"]

for (ax, label, print_label) in zip(axs.flat, aggr_collision_names, translation):
    for (table, bucket_size) in zip(stats_dfs, bucket_numbers):
        ser = pd.Series([0] * entries_per_rank)
        for rank in range(0, rank_count):
            local = table.loc[table['rank'] == rank]
            l_cols = local[label]

            l_cols.reset_index(drop=True, inplace=True)
            ser = np.maximum(ser, l_cols)
        
        ax.plot(ser, label=str(bucket_size))
        
    if print_label == 'MPI_ANY_SRC':
        ax.yaxis.set_major_locator(ticker.MultipleLocator(50))
        ax.yaxis.set_minor_locator(ticker.MultipleLocator(25))
    else:
        ax.yaxis.set_major_locator(ticker.MultipleLocator(10))
        ax.yaxis.set_minor_locator(ticker.MultipleLocator(5))

    ax.set_xticks([])
    ax.set_title(print_label, fontsize=10)
    ax.grid(True)


fig.supxlabel('Runtime')
fig.supylabel('Max. number of collisions')
fig.legend(labels=[str(i) for i in bucket_numbers], title='N. Bins', loc='lower right') 

plt.suptitle(app_name)

plt.tight_layout()
plt.show()

In [ ]:
# Min. perc. of empty bins
fig, axs = plt.subplots(3, 1, sharex=True, sharey=False)

translation = ["No Wildcard", "MPI_ANY_SRC", "MPI_ANY_TAGminiFE"]

for (ax, label, print_label) in zip(axs.flat, collision_prob_names, translation):
    for (table, bucket_size) in zip(stats_dfs, bucket_numbers):
        ser = pd.Series([1.0] * entries_per_rank)
        for rank in range(0, rank_count):
            local = table.loc[table['rank'] == rank]
            l_cols = local[label]

            l_cols.reset_index(drop=True, inplace=True)
            ser = np.minimum(ser, l_cols)
        
        ax.plot(ser, label=str(bucket_size))
       
    if print_label == 'MPI_ANY_SRC':
        ax.yaxis.set_major_locator(ticker.MultipleLocator(0.0125))
        ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.00625))
        ax.yaxis.set_major_formatter(PercentFormatter(1.00, decimals=0))
    else:
        ax.yaxis.set_major_locator(ticker.MultipleLocator(0.25))
        ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.05))
        ax.yaxis.set_major_formatter(PercentFormatter(1.00, decimals=0))

    ax.set_xticks([])
    ax.set_title(print_label, fontsize=10)
    ax.grid(True)


fig.supxlabel('Runtime')
fig.supylabel('%. of empty buckets')
fig.legend(labels=[str(i) for i in bucket_numbers], title='N. Bins', loc='lower right') 

plt.suptitle(app_name)

plt.tight_layout()
plt.show()

In [ ]:
# Aggr. %. of collisions per bin
x = list(range(0, entries_per_rank))
fig, axs = plt.subplots(3, 1, sharex=True, sharey=True)

for (ax, label, count_labels) in zip(axs.flat, collision_names, recvs_count_names):
    for (table, bucket_size) in zip(stats_dfs, bucket_numbers):
        ser = pd.Series([0] * entries_per_rank)
        total_ser = pd.Series([0] * entries_per_rank)

        # Count total number of recvs
        for rank in range(0, rank_count):
            local = table.loc[table['rank'] == rank]
            l_cols = local[label]

            l_cols.reset_index(drop=True, inplace=True)

            ser += l_cols
            total_ser += local[count_labels].reset_index().sum(axis=1)
        
        ax.plot(x, ser / total_ser / precision, label=str(bucket_size))

    ax.set_ylim(bottom=0)
    ax.set_xticks([])
    ax.set_title(label, fontsize=10)
    ax.grid(True)


fig.supxlabel('Runtime')
fig.supylabel('% number of collisions aggr. ranks')
fig.legend(labels=[str(i) for i in bucket_numbers], title='N. Bins', loc='lower right') 

plt.suptitle("% number of collisions by bin sizes over time")

plt.tight_layout()
plt.show()

In [ ]:
# Aggr. Avg. of collisions per bin
fig, axs = plt.subplots(1, 3, sharex=False, sharey=True)

for (ax, label) in zip(axs.flat, collision_names):
    ax.set_title(label, fontsize=10)
    
    bar_data = list()

    for (table, bucket_size) in zip(stats_dfs, bucket_numbers):
        ser = pd.Series([0] * entries_per_rank)
        for rank in range(0, rank_count):
            local = table.loc[table['rank'] == rank]
            l_cols = local[label]

            l_cols.reset_index(drop=True, inplace=True)
            ser += l_cols
        
        bar_data.append((ser / rank_count).sum() / entries_per_rank / precision)

    ax.bar([str(i) for i in bucket_numbers], bar_data)
    
fig.supxlabel('Number of bins')
fig.supylabel('Avg. number of collisions')

plt.suptitle("Avg. number of collisions aggr. ranks over time", fontsize=18)

plt.tight_layout()
plt.show()

In [ ]:
# Max. queue depth over time
fig, axs = plt.subplots(4, 1, sharex=True, sharey=True)

for (ax, label) in zip(axs.flat, collision_depth_names):
    for (table, bucket_size) in zip(stats_dfs, bucket_numbers):
        ser = pd.Series([0] * entries_per_rank)
        for rank in range(0, rank_count):
            local = table.loc[table['rank'] == rank]
            l_cols = local[label]

            l_cols.reset_index(drop=True, inplace=True)
            ser = np.maximum(ser, l_cols)
        
        ax.plot(ser, label=str(bucket_size))

    ax.set_ylim(bottom=0)
    ax.set_xticks([])
    ax.set_title(label, fontsize=1S: 1
fig.supylabel('Max. queue depth')
fig.legend(labels=[str(i) for i in bucket_numbers], title='N. Bins', loc='lower right') 

plt.suptitle("Max. queue depth over time aggr. ranks by bin size")

plt.tight_layout()
plt.show()

In [ ]:
# Tag distribution over time aggr. all ranks
fig, axs = plt.subplots(4, 1, sharex=True, sharey=False)
tag_op = ['send', 'recv']
tag_kind = ['sync', 'async']

# tag_table = tags_dfs[0]
tag_table = tags_dfs

used_tags = set()

for (ax, (op, kind)) in zip(axs.flat, itertools.product(tag_op, tag_kind)):
    filter_op_kind = tag_table.loc[(tag_table['op'] == op) & (tag_table['kind'] == kind)]

    acc_df = pd.DataFrame()
    for rank in range(0, rank_count):
        filter_data = filter_op_kind.loc[tag_table['rank'] == rank]

        if acc_df.empty:
            acc_df = filter_data.iloc[:,3:].reset_index(drop=True)
        else:
            acc_df += filter_data.iloc[:,3:].reset_index(drop=True)

        for c in acc_df.columns:
            used_tags.add(int(c))

    if not acc_df.empty:
    #     # for col in acc_df.columns:
    #     #     acc_df[col] /= acc_df[col].sum()
        ax.plot(acc_df)

    ax.set_title(op + '_' + kind)
    ax.set_ylim(bottom=0)
    ax.set_xticks([])
    ax.grid(True)
    ax.yaxis.set_major_locator(MaxNLocator(nbins='3', integer=True))
        
fig.supxlabel('Runtime')
fig.supylabel('Total number of messages')
if len(used_tags) < 6:
    fig.legend(sorted(used_tags), title="Tag")

plt.suptitle("Tag distribution aggr. ranks", fontsize=18)

plt.tight_layout()
plt.show()

print('Unique TAGS: ' + str(len(used_tags)))